In [33]:
# load csv
import pandas as pd
df = pd.read_csv('results_test.csv')
df.head()

,start,end,text,label,score
0,2,19,CAMERON HIGHLANDS,LOC,0.892983
1,32,49,Cameron Highlands,LOC,0.917256
2,307,334,Perintah Kawalan Pergerakan,EVENT,0.654827
3,419,428,strawberi,PRODUCT,0.800663
4,430,446,pakaian terpakai,PRODUCT,0.723930


## Q1. Label Distribution

In [34]:
# drop start, end, and score columns
q1 = df.drop(['start', 'end', 'score'], axis = 1)
q1.head()

,text,label
0,CAMERON HIGHLANDS,LOC
1,Cameron Highlands,LOC
2,Perintah Kawalan Pergerakan,EVENT
3,strawberi,PRODUCT
4,pakaian terpakai,PRODUCT


In [35]:
# normalize text
q1['text'] = q1['text'].str.lower()
q1.head()

,text,label
0,cameron highlands,LOC
1,cameron highlands,LOC
2,perintah kawalan pergerakan,EVENT
3,strawberi,PRODUCT
4,pakaian terpakai,PRODUCT


In [ ]:
# duplicates
q1.drop_duplicates().head()

,text,label
0,cameron highlands,LOC
2,perintah kawalan pergerakan,EVENT
3,strawberi,PRODUCT
4,pakaian terpakai,PRODUCT
5,cenderamata,PRODUCT


In [ ]:
# label distribution
q1['label'].value_counts()

label
LOC        5
PRODUCT    5
PERSON     3
TIME       3
EVENT      2
Name: count, dtype: int64